In [85]:
import numpy as np
import ast, dataIO, common_functions
from sklearn import cross_validation, linear_model, preprocessing, svm
from sklearn.grid_search import GridSearchCV
reload(dataIO)
reload(common_functions)

postures = {"left_hand":["4", "8", "11"], "right_hand":["1", "7", "10"], 
            "index_finger":["3", "5", "12"], "two_hand":["2", "6", "9"]}

locations = []
bod = []
targets_x = []
targets_y = []
mse_x = []
mse_y = []

def process_twohand(userid, filenos):
    
    locations.append([])
    bod.append([])
    targets_x.append([])
    targets_y.append([])
    locations.append([])
    bod.append([])
    targets_x.append([])
    targets_y.append([])
    se_x = [[],[]] 
    se_y = [[],[]]
    centers = dataIO.get_key_centers()
    index_l = len(locations)-2
    index_r = len(locations)-1
    
    for fileno in filenos:
        filename = "/home/dimitar/Desktop/Python/experiment/results/"+str(userid)+"_"+fileno+"up.txt"
        with open(filename, "r") as f:
            lines = f.read().splitlines()
            lines = map(lambda x: x.split('\t'), lines[1:])
            for line in lines:
                letter = line[1]
                location = list(ast.literal_eval(line[3]))
                center = centers[letter]
                
                if not dataIO.iscorrect(location, center):
                    continue
                
                offset_x = center[0]-location[0]
                offset_y = center[1]-location[1]
                if line[0] == "left":
                    index = index_l
                    se_x[0].append(offset_x**2)
                    se_y[0].append(offset_y**2)
                else:
                    index = index_r
                    se_x[1].append(offset_x**2)
                    se_y[1].append(offset_y**2)                
                
                targets_x[index].append(offset_x)
                targets_y[index].append(offset_y)                
                locations[index].append(location)
                bod[index].append(dataIO.createlist(line[-1]))
                
    mse_x.append(np.mean(np.array(se_x[0])))
    mse_y.append(np.mean(np.array(se_y[0])))
    mse_x.append(np.mean(np.array(se_x[1])))
    mse_y.append(np.mean(np.array(se_y[1]))) 
    

def process_posture(userid, filenos):
    
    locations.append([])
    bod.append([])
    targets_x.append([])
    targets_y.append([])
    se_x= [] 
    se_y = []
    centers = dataIO.get_key_centers()
    index = len(locations)-1
    
    for fileno in filenos:
        filename = "/home/dimitar/Desktop/Python/experiment/results/"+str(userid)+"_"+fileno+"up.txt"
        with open(filename, "r") as f:
            lines = f.read().splitlines()
            lines = map(lambda x: x.split('\t'), lines[1:])
            for line in lines:
                letter = line[0]
                location = list(ast.literal_eval(line[2]))
                center = centers[letter]
                
                if not dataIO.iscorrect(location, center):
                    continue
                
                offset_x = center[0]-location[0]
                offset_y = center[1]-location[1]
                targets_x[index].append(offset_x)
                targets_y[index].append(offset_y)
                se_x.append(offset_x**2)
                se_y.append(offset_y**2)
                
                locations[index].append(location)
                bod[index].append(dataIO.createlist(line[-1]))
                
    mse_x.append(np.mean(np.array(se_x)))
    mse_y.append(np.mean(np.array(se_y)))                

def read_data(userid):    
    keys = postures.keys()
    
    for key in keys:
        filenos = postures[key]
        if key=="two_hand":
            process_twohand(userid, filenos)
        else:
            process_posture(userid, filenos)

read_data(3)

#y = [i for i in range(len(locations)) for x in range(len(locations[i]))]

i = 4
print np.array(locations[i]).shape
print np.array(bod[i]).shape
print np.array(targets_x[i]).shape
print np.array(targets_y[i]).shape

print("MSE on x before regression: %.2f" % np.mean(mse_x))
print("MSE on y before regression: %.2f" % np.mean(mse_y))

points_flat = [x for sublist in locations for x in sublist]


kf = cross_validation.KFold(len(y), n_folds=10, shuffle=True)

for train_index, test_index in kf:
    points_flat = [x for sublist in locations for x in sublist]
    
    points_train, points_test = locations[train_index], locations[test_index]
    bod_train, bod_test = locations[train_index], locations[test_index] 



(154, 2)
(154, 24)
(154,)
(154,)
MSE on x before regression: 162.68
MSE on y before regression: 131.15
772


TypeError: only integer arrays with one element can be converted to an index

In [110]:
import numpy as np
import ast, dataIO, common_functions
from sklearn import cross_validation, linear_model, preprocessing, svm
from sklearn.grid_search import GridSearchCV
reload(dataIO)
reload(common_functions)

postures = {"left_hand":["4", "8", "11"], "right_hand":["1", "7", "10"], 
            "index_finger":["3", "5", "12"], "two_hand":["2", "6", "9"]}

locations = []
bod = []
targets_x = []
targets_y = []
y = []
mse_x = []
mse_y = []

def learn_offset(points, targets):
    regr = linear_model.LinearRegression()
    regr.fit(points, targets)
    
    return regr

def process_twohand(userid, filenos, posture):
    
    se_x = [[],[]] 
    se_y = [[],[]]
    centers = dataIO.get_key_centers()
    
    for fileno in filenos:
        filename = "/home/dimitar/Desktop/Python/experiment/results/"+str(userid)+"_"+fileno+"up.txt"
        with open(filename, "r") as f:
            lines = f.read().splitlines()
            lines = map(lambda x: x.split('\t'), lines[1:])
            for line in lines:
                letter = line[1]
                location = list(ast.literal_eval(line[3]))
                center = centers[letter]
                
                if not dataIO.iscorrect(location, center):
                    continue
                
                offset_x = center[0]-location[0]
                offset_y = center[1]-location[1]
                if line[0] == "left":
                    se_x[0].append(offset_x**2)
                    se_y[0].append(offset_y**2)
                    y.append(posture)
                else:
                    se_x[1].append(offset_x**2)
                    se_y[1].append(offset_y**2)
                    y.append(posture+1)
                
                targets_x.append(offset_x)
                targets_y.append(offset_y)                
                locations.append(location)
                bod.append(dataIO.createlist(line[-1]))
                
    mse_x.append(np.mean(np.array(se_x[0])))
    mse_y.append(np.mean(np.array(se_y[0])))
    mse_x.append(np.mean(np.array(se_x[1])))
    mse_y.append(np.mean(np.array(se_y[1]))) 
    

def process_posture(userid, filenos, posture):    
    se_x = [] 
    se_y = []
    centers = dataIO.get_key_centers()
    
    for fileno in filenos:
        filename = "/home/dimitar/Desktop/Python/experiment/results/"+str(userid)+"_"+fileno+"up.txt"
        with open(filename, "r") as f:
            lines = f.read().splitlines()
            lines = map(lambda x: x.split('\t'), lines[1:])
            for line in lines:
                letter = line[0]
                location = list(ast.literal_eval(line[2]))
                center = centers[letter]
                
                if not dataIO.iscorrect(location, center):
                    continue
                
                offset_x = center[0]-location[0]
                offset_y = center[1]-location[1]
                targets_x.append(offset_x)
                targets_y.append(offset_y)
                se_x.append(offset_x**2)
                se_y.append(offset_y**2)
                
                locations.append(location)
                bod.append(dataIO.createlist(line[-1]))
                y.append(posture)
                
    mse_x.append(np.mean(np.array(se_x)))
    mse_y.append(np.mean(np.array(se_y)))                

def read_data(userid):    
    keys = postures.keys()
    
    posture = 0
    for key in keys:
        filenos = postures[key]
        if key=="two_hand":
            process_twohand(userid, filenos, posture)
            posture += 2
        else:
            process_posture(userid, filenos, posture)
            posture += 1

read_data(12)

print("MSE on x before regression: %.2f" % np.mean(mse_x))
print("MSE on y before regression: %.2f" % np.mean(mse_y))

locations = np.array(locations)
bod = np.array(bod)
targets_x = np.array(targets_x)
targets_y = np.array(targets_y)
y = np.array(y)

foldno = 1
mse_x = []
mse_y = []
kf = cross_validation.KFold(len(y), n_folds=10, shuffle=True)

for train_index, test_index in kf:    
    points_train, points_test = locations[train_index], locations[test_index]
    bod_train, bod_test = bod[train_index], bod[test_index]
    t_x_train, t_x_test = targets_x[train_index], targets_x[test_index]
    t_y_train, t_y_test = targets_y[train_index], targets_y[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    scaler = preprocessing.StandardScaler().fit(bod_train)  
    bod_scaled = scaler.transform(bod_train)
    
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1, 0.1 ,1e-2, 1e-3, 1e-4],
                         'C': [0.001, 0.01, 0.1 ,1, 10, 100, 1000]},
                        {'kernel': ['linear'], 'C': [0.001, 0.01, 0.1 ,1, 10, 100, 1000]}]
    
    clf = GridSearchCV(svm.SVC(C=1, cache_size=500), tuned_parameters)
    clf.fit(bod_scaled, y_train)
    
    print clf.best_estimator_
    print
    
    regr_x = []
    regr_y = []
    
    for i in range(0,5):
        index = np.where(y_train==i)[0]
        regr_x.append(learn_offset(points_train[index], t_x_train[index]))
        regr_y.append(learn_offset(points_train[index], t_y_train[index]))
        
    se_x = []
    se_y = []
    for i in range(len(points_test)):
        
        point = points_test[i]
        bod_data = scaler.transform(bod_test[i])
        pred = clf.predict(bod_data)
        
        pred_x = regr_x[pred].predict(point)
        pred_y = regr_y[pred].predict(point)
        
        se_x.append((pred_x - t_x_test[i])**2)
        se_y.append((pred_y - t_y_test[i])**2)
        
    mse = np.mean(np.array(se_x))
    print("MSE on x: %.2f" %mse)
    mse_x.append(mse)
    
    mse = np.mean(np.array(se_y))
    print("MSE on y: %.2f" %mse)
    mse_y.append(mse)
    
    foldno +=1    


print 
print("Mean MSE on on x: %.2f" % np.mean(mse_x))
print("Mean MSE on on y: %.2f" % np.mean(mse_y))
        
        
    

MSE on x before regression: 311.55
MSE on y before regression: 122.84
SVC(C=1000, cache_size=500, class_weight=None, coef0=0.0, degree=3,
  gamma=0.01, kernel='rbf', max_iter=-1, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

MSE on x: 89.01
MSE on y: 97.03
SVC(C=1000, cache_size=500, class_weight=None, coef0=0.0, degree=3,
  gamma=0.01, kernel='rbf', max_iter=-1, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

MSE on x: 83.00
MSE on y: 92.49
SVC(C=100, cache_size=500, class_weight=None, coef0=0.0, degree=3, gamma=0.01,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

MSE on x: 87.82
MSE on y: 90.46
SVC(C=10, cache_size=500, class_weight=None, coef0=0.0, degree=3, gamma=0.01,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

MSE on x: 62.96
MSE on y: 80.65
SVC(C=100, cache_size=500, class_weig